### In this script we will extract metrics for each commit. For that you need to download and install Understand Tool, available here https://scitools.com/trial-download-3/. You need to install in 'SANER2022-CodeSamples/2-ExtractingMetrics/understand'. To run you will need license to Understand, student get free in https://scitools.com/non-commercial-license/.

In [1]:
pip install GitPython

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install JPype1

In [3]:
from git import Repo
import pandas as pd
import numpy as np
import os
import fnmatch
from statistics import mean
import shutil

In [4]:
def printStatus(index, size):
    print("{0}% Completed samples".format((index / size) * 100))

In [5]:
def checkoutTo(project, sha):
    projectPath = "repositories/" + project
    repository = Repo(projectPath)
    repository.git.checkout(sha, "-f")

In [6]:
def extractMetricsWithUnderstand(owner, project):
    understandPath = "understand\\SciTools\\bin\\pc-win64\\und"
    os.system('cmd /c "{0} create -languages java {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} add {1} {2}"'.format(understandPath, "repositories/"+owner+"/"+project, project))
    os.system('cmd /c "{0} settings -metrics all {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} settings -metricsOutputFile {1}.csv {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} -quiet analyze {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} metrics {1}"'.format(understandPath, project))

In [7]:
def class_level_metrics(dataframe, owner, project):
    dataframe = dataframe[dataframe["Kind"].str.contains("Class")]
    dataframe["sample"] = owner+"/"+project
    totalLOC = dataframe["CountLine"].sum()
    dataframe = dataframe.groupby("sample").mean()
    dataframe["CountTotalLine"] = totalLOC
    return dataframe

In [8]:
def project_level_metrics(dataframe, owner, project):
    numberJavaFiles = 0 if dataframe.empty else dataframe["Kind"].value_counts()["File"]
    dataframe = dataframe[dataframe["Kind"] == "Package"]
    packages = len(dataframe)
    dataframe.drop(dataframe.columns.difference(['CountDeclClass']), 1, inplace=True)
    dataframe["sample"] = owner+"/"+project
    dataframe = dataframe.groupby("sample").sum()
    dataframe["JavaFiles"] = numberJavaFiles
    dataframe["Packages"] = packages
    return dataframe

In [9]:
def readMetricsFromCsv(project):
    dtype = {'Kind': str, 'Name': str, 'AvgCyclomatic': np.float64, 'AvgCyclomaticModified': np.float64,'AvgCyclomaticStrict': np.float64, 'AvgEssential': np.float64, 'AvgLine': np.float64, 'AvgLineBlank': np.float64,'AvgLineCode': np.float64, 'AvgLineComment': np.float64, 'CountClassBase': np.float64, 'CountClassCoupled': np.float64,'CountClassCoupledModified': np.float64, 'CountClassDerived': np.float64, 'CountDeclClass': np.float64,'CountDeclClassMethod': np.float64, 'CountDeclClassVariable': np.float64,'CountDeclExecutableUnit': np.float64, 'CountDeclFile': np.float64, 'CountDeclFunction': np.float64,'CountDeclInstanceMethod': np.float64, 'CountDeclInstanceVariable': np.float64,'CountDeclMethod': np.float64, 'CountDeclMethodAll': np.float64, 'CountDeclMethodDefault': np.float64,'CountDeclMethodPrivate': np.float64, 'CountDeclMethodProtected': np.float64,'CountDeclMethodPublic': np.float64, 'CountInput': np.float64,'CountLine': np.float64, 'CountLineBlank': np.float64,'CountLineCode': np.float64, 'CountTotalLine': np.float64, 'CountLineCodeDecl': np.float64, 'CountLineCodeExe': np.float64,'CountLineComment': np.float64, 'CountOutput': np.float64, 'CountPath': np.float64, 'CountPathLog': np.float64,'CountSemicolon': np.float64, 'CountStmt': np.float64, 'CountStmtDecl': np.float64, 'CountStmtExe': np.float64,'Cyclomatic': np.float64, 'CyclomaticModified': np.float64, 'CyclomaticStrict': np.float64, 'Essential': np.float64, 'Knots': np.float64, 'MaxCyclomatic': np.float64, 'MaxCyclomaticModified': np.float64,'MaxCyclomaticStrict': np.float64, 'MaxEssential': np.float64,'MaxEssentialKnots': np.float64,'MaxInheritanceTree': np.float64, 'MaxNesting': np.float64, 'MinEssentialKnots': np.float64,'PercentLackOfCohesion': np.float64, 'PercentLackOfCohesionModified': np.float64,'RatioCommentToCode': np.float64, 'SumCyclomatic': np.float64, 'SumCyclomaticModified': np.float64,'SumCyclomaticStrict': np.float64, 'SumEssential': np.float64}
    dataframe = pd.read_csv(project+".csv", dtype=dtype)
    return dataframe

In [10]:
def getUnderstandMetrics(owner, project):
    extractMetricsWithUnderstand(owner, project)
    
    dataframe = readMetricsFromCsv(project)
        
    class_level = class_level_metrics(dataframe.copy(), owner, project)
    class_level["level"] = "class"
    class_level.set_index("level")    
    
    project_level = project_level_metrics(dataframe.copy(), owner, project)
    project_level["level"] = "project"
    project_level.set_index("level")
    
    return pd.concat([class_level, project_level], axis=0)

In [11]:
def findPaths(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        if '.git' in root:
            continue
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

In [12]:
def getMetrics(owner, project):
    metrics = getUnderstandMetrics(owner, project)
    return metrics

In [13]:
def deleteUnusedFiles(sample):
    os.remove(sample + ".udb")
    os.remove(sample + ".csv")    

In [14]:
def createDirectoryIfNotExists(dirName):
    if not os.path.exists(dirName):
            os.makedirs(dirName)

In [15]:
def extractMetricsByCommit(sample):
    owner, project = sample.split("/")
    metrics = getMetrics(owner, project)
    return metrics

In [16]:
def extractMetricsForAllCommits(commits, sample):
    allCommits = pd.DataFrame()
    for index, commit in enumerate(commits):
        metrics = extractMetricsByCommit(sample, commit)
        allCommits = allCommits.append(metrics, ignore_index=True)
        print("{0}% of commits completed from sample {1}".format((index/len(commits) * 100), sample))
        allCommits.to_csv("metrics\\"+sample+".csv", index=False)
    return allCommits

In [17]:
def clone(gitUrl, repoDir, sample):
    Repo.clone_from(gitUrl, repoDir + sample)

In [18]:
def download(sample):
    gitHubUrl = createGitHubUrl(sample)
    print("Downloading %s" % sample)
    repoDir = "repositories/"
    isdir = os.path.isdir(repoDir+sample)
    if isdir:
        print("Project " + sample + " previously downloaded")
    else:
        clone(gitHubUrl, repoDir, sample)
        print("%s downloaded" % sample)

In [19]:
def createGitHubUrl(sample):
    return "https://github.com/" + sample + ".git"

In [20]:
def metricsByCommit(inputs):
    
    createDirectoryIfNotExists("repositories")
    createDirectoryIfNotExists("metrics")
    exclude = []
    
    last_index = inputs.index[-1]
    
    for index in inputs.index:
        
        input_row = inputs.loc[index]
        repository = input_row["watcher_repo"]
        
        printStatus(index, last_index)
        try:
            download(repository)
        except:
            continue
        
        owner = repository.split("/")[0]
        repo = repository.split("/")[1]
        
        createDirectoryIfNotExists(f"repositories/{owner}")
        createDirectoryIfNotExists(f"metrics/{owner}")
        
        metrics = extractMetricsByCommit(repository)

        metrics.to_csv("metrics/"+repository+".csv", index=False)
        
        deleteUnusedFiles(f"{repo}")

In [21]:
samples = pd.read_csv("../5-RepoFilter/output.csv", index_col=0)

In [22]:
samples

,framework,sample,watcher,watcher_repo
21534,googlearchive,googlearchive/android-instant-apps,xiangjiaowangzi,xiangjiaowangzi/TransitionDemo
23797,googlearchive,googlearchive/leanback-assistant,yuliskov,yuliskov/MyVideoTube
23800,googlearchive,googlearchive/leanback-assistant,yuliskov,yuliskov/SmartTubeNext
37953,googlearchive,googlearchive/android-SynchronizedNotifications,morristech,morristech/ModuleLoader
38182,googlearchive,googlearchive/android-SynchronizedNotifications,tun-lin,tun-lin/HappyShop
...,...,...,...,...
374083,googlearchive,googlearchive/android-credentials,morristech,morristech/Android-OCR
374233,googlearchive,googlearchive/android-credentials,morristech,morristech/Time-partner
374256,googlearchive,googlearchive/android-credentials,thucld,thucld/Choukai-Tasuku
374426,googlearchive,googlearchive/android-credentials,LastSocialDroid2,LastSocialDroid2/MapTest


In [23]:
lista = samples["watcher_repo"].tolist()
for i in range(len(lista)):
    sample = lista[i]
    if sample == "jkYishon/android6.0":
        print(i)

40
448
648


In [24]:
# samples = samples[40:]

In [25]:
metricsByCommit(samples)

21.36120540575162% Completed samples
Project jkYishon/android6.0 previously downloaded


C:\Users\mrgab\AppData\Local\Temp/ipykernel_5672/1567332605.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["sample"] = owner+"/"+project
C:\Users\mrgab\AppData\Local\Temp/ipykernel_5672/379348769.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataframe.drop(dataframe.columns.difference(['CountDeclClass']), 1, inplace=True)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


21.45407673092935% Completed samples
Project xdtianyu/android_2.3.5 previously downloaded
21.58404321185337% Completed samples
Project lmjssjj/wiki previously downloaded
21.66090223958667% Completed samples
Project jalong4/AttestProps previously downloaded
21.662770340955188% Completed samples
Project jalong4/UniqueID previously downloaded
21.718012767138497% Completed samples
Project ryuunoakaihitomi/PowerAct previously downloaded
22.26403210999381% Completed samples
Project CaMnter/AndroidLife previously downloaded
23.3584726403211% Completed samples
Project 54binge/EasySubway previously downloaded
23.75637823181537% Completed samples
Project 0xm1nam0/RxCore previously downloaded
24.284517175857726% Completed samples
Project m122469119/EcxppSDK previously downloaded
24.385661521381756% Completed samples
Project qyxxjd/AndroidBasicProject previously downloaded
24.38592839300583% Completed samples
Project qyxxjd/BaseProject previously downloaded
24.462787420739126% Completed samples
Pr

38.41136659621256% Completed samples
Project wax911/Vision-Barcode-Scanner previously downloaded
38.45513354256069% Completed samples
Project prathanbomb/mlkit-vision-quickstart previously downloaded
38.480219475223635% Completed samples
Project zhorizon/SuperDuo previously downloaded
38.483155063088454% Completed samples
Project hbaxamoosa/alexandria previously downloaded
38.518915860714365% Completed samples
Project kuanyingchou/udacity-barcode previously downloaded
38.52532077969214% Completed samples
Project miguelcamposdev/pmdm previously downloaded
38.53572877303102% Completed samples
Project mananwason/Re.Wise previously downloaded
38.586167509981% Completed samples
Project IhorKlimov/Alexandria_Udacity previously downloaded
38.60431478041802% Completed samples
Project jbc25/boniatillo previously downloaded
38.619526462990244% Completed samples
Project yatatsu/PrettyQR previously downloaded
38.6302013279532% Completed samples
Project Hatem-tmi/food-facts previously downloaded
38

40.58370161617456% Completed samples
Project Kevin-Mok/ParsaFood previously downloaded
40.59704519737825% Completed samples
Project NoahSchaub/Rezepteverwaltung previously downloaded
40.59731206900233% Completed samples
Project NoahSchaub/Rezepteverwaltung_2 previously downloaded
40.64214650184675% Completed samples
Project RajPrabhakar/Pointer previously downloaded
40.65202075193749% Completed samples
Project TWJessieChen/Camera1Camera2Combined previously downloaded
40.65228762356156% Completed samples
Project TWJessieChen/CameraMediacodec previously downloaded
40.66029377228378% Completed samples
Project rohan23chhabra/Oculus-Apertor previously downloaded
40.66696556288563% Completed samples
Project csanjeev25/SafeDrive previously downloaded
40.685646576570804% Completed samples
Project tssA8/camerawithGoogleVisionApi previously downloaded
40.73581844189671% Completed samples
Project kolibreath/JoinMe previously downloaded
40.75876940156707% Completed samples
Project shakram02/QRra2 

...

74.52390102265205% Completed samples
Project liqy/Hello-RxJava previously downloaded
74.57327227310574% Completed samples
Project dejavu1988/Android-SyncAdapterDemo previously downloaded
74.64986442921497% Completed samples
Project Leaking/Android_Learning_Demo previously downloaded
74.66641046990755% Completed samples
Project owenlwg/AndroidProgramGuide previously downloaded
74.70270501078161% Completed samples
Project ChenQingZhen/BluetoothProj previously downloaded
74.81185550502786% Completed samples
Project ha271923/BluetoothBLEChat previously downloaded
74.81292299152416% Completed samples
Project ha271923/MyBasicAppComponents previously downloaded
74.95623305365187% Completed samples
Project ainiyiwan/GoogleNotificationStudy previously downloaded
75.05284058156664% Completed samples
Project NatureDay/DisplayingBitmaps previously downloaded
75.14304319050365% Completed samples
Project iamliujie1023/myDemo previously downloaded
75.18947885309251% Completed samples
Project Ericliu0

78.09651145413011% Completed samples
Project CCCCCoco/Android-RoadRollerUpper previously downloaded
78.13360660987638% Completed samples
Project l1d000/AndroidBox previously downloaded
78.13387348150046% Completed samples
Project l1d000/CurrentBLEGatt previously downloaded
78.17283673861526% Completed samples
Project Leekyuhwa/android-CustomChoiceList-master previously downloaded
78.21366809709858% Completed samples
Project leerduo/ImmersiveMode previously downloaded
78.21713742821153% Completed samples
Project leerduo/MyDragLayout previously downloaded
78.2206067593245% Completed samples
Project leerduo/OfficialSwipeRefreshLayout previously downloaded
78.22274173231709% Completed samples
Project leerduo/RecyclerViewDemo previously downloaded
78.2256773201819% Completed samples
Project leerduo/thinking previously downloaded
78.24542582036337% Completed samples
Project TonyTangAndroid/SlidingTabsBasic previously downloaded
78.2739810841393% Completed samples
Project morristech/coinbase-

80.3611840560217% Completed samples
Project djoly42/android-chat-bluetooth previously downloaded
80.36145092764576% Completed samples
Project djoly42/car-android-app-panel previously downloaded
80.36225154251798% Completed samples
Project djoly42/road-sign-detection previously downloaded
80.36892333311984% Completed samples
Project konatsup/AndroidRPG previously downloaded
80.394809880655% Completed samples
Project liuzhao1006/Bluetoothss previously downloaded
80.43163816477723% Completed samples
Project cereuz/android-BluetoothChat previously downloaded
80.43804308375499% Completed samples
Project Traceless2016/BluetoothCarDemo previously downloaded
80.50929780738274% Completed samples
Project hendrawd/drag-fast-scroller-test previously downloaded
80.51650334123273% Completed samples
Project bloomlj/wildrobot previously downloaded
80.52477636157903% Completed samples
Project taynanbonaldo/WrappedBluetoothMessage previously downloaded
80.54986229424198% Completed samples
Project MrXion

86.56408121437264% Completed samples
Project raghunandankavi2010/SamplesAndroid previously downloaded
86.56568244411709% Completed samples
Project xiaomaguoguo/android-DisplayingBitmaps previously downloaded
86.56674993061337% Completed samples
Project xiaomaguoguo/CustomRecycleview previously downloaded
86.60037575524669% Completed samples
Project subinkrishna/SlidingTabsBasicSample previously downloaded
86.72740664830589% Completed samples
Project morristech/ModuleLoader previously downloaded
86.84456329127437% Completed samples
Project hzh-romens/Health previously downloaded
87.03751147547983% Completed samples
Project Piashsarker/DirectShareForMarshmallow previously downloaded
87.08261277994833% Completed samples
Project mohanrajsambath/android-DirectShare-1 previously downloaded
87.16614359828348% Completed samples
Project philburk/android-midisuite previously downloaded
87.17761907811867% Completed samples
Project kshoji/Android-MIDI-API-backports previously downloaded
87.3310702

96.68225196951259% Completed samples
Project lsjwzh/DownloadableFontsProvider previously downloaded
96.79754051111253% Completed samples
Project clwater/EmojiKeyboard previously downloaded
98.87193364503939% Completed samples
Project morristech/ModuleLoader previously downloaded
98.92237238198936% Completed samples
Project baoyongzhang/BeanPaste previously downloaded
98.95146138901343% Completed samples
Project xiaomaguoguo/android-DisplayingBitmaps previously downloaded
98.95252887550973% Completed samples
Project xiaomaguoguo/CustomRecycleview previously downloaded
98.98321911227823% Completed samples
Project subinkrishna/SlidingTabsBasicSample previously downloaded
99.04673455880784% Completed samples
Project tslxt/Time-Card previously downloaded
99.09583893763744% Completed samples
Project morristech/ModuleLoader previously downloaded
99.22286983069665% Completed samples
Project morristech/ModuleLoader previously downloaded
99.25996498644292% Completed samples
Project marti4ka/glow